In [27]:
import pandas as pd 
import numpy as np
import seaborn as sns
import category_encoders as ce

import matplotlib.pyplot as plt 
%matplotlib inline

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, log_loss, matthews_corrcoef, balanced_accuracy_score



In [28]:
# Đọc file CSV
df = pd.read_csv('Traffic_Crashes_-_Crashes.csv')

In [29]:
df.describe()

,POSTED_SPEED_LIMIT,LANE_CNT,STREET_NO,BEAT_OF_OCCURRENCE,NUM_UNITS,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE
count,794956.000000,1.990060e+05,794956.000000,794951.000000,794956.000000,793215.000000,793215.000000,793215.000000,793215.000000,793215.000000,793215.000000,793215.0,794956.000000,794956.000000,794956.000000,789567.000000,789567.000000
mean,28.402547,1.333032e+01,3688.085435,1242.916397,2.034919,0.189448,0.001195,0.020032,0.107025,0.061195,2.004313,0.0,13.202800,4.122344,6.787978,41.854788,-87.673450
std,6.179272,2.961623e+03,2887.990566,705.268208,0.452475,0.565654,0.037456,0.165673,0.422095,0.318451,1.158715,0.0,5.568727,1.980411,3.405398,0.337523,0.686153
min,0.000000,0.000000e+00,0.000000,111.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,1.000000,0.000000,-87.936193
25%,30.000000,2.000000e+00,1248.000000,714.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,9.000000,2.000000,4.000000,41.782429,-87.721709
50%,30.000000,2.000000e+00,3201.000000,1211.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.0,14.000000,4.000000,7.000000,41.874743,-87.674094
75%,30.000000,4.000000e+00,5600.000000,1822.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.0,17.000000,6.000000,10.000000,41.924336,-87.633300
max,99.000000,1.191625e+06,451100.000000,6100.000000,18.000000,21.000000,4.000000,10.000000,21.000000,15.000000,61.000000,0.0,23.000000,7.000000,12.000000,42.022780,0.000000


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 794956 entries, 0 to 794955
Data columns (total 48 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                794956 non-null  object 
 1   CRASH_DATE_EST_I               59606 non-null   object 
 2   CRASH_DATE                     794956 non-null  object 
 3   POSTED_SPEED_LIMIT             794956 non-null  int64  
 4   TRAFFIC_CONTROL_DEVICE         794956 non-null  object 
 5   DEVICE_CONDITION               794956 non-null  object 
 6   WEATHER_CONDITION              794956 non-null  object 
 7   LIGHTING_CONDITION             794956 non-null  object 
 8   FIRST_CRASH_TYPE               794956 non-null  object 
 9   TRAFFICWAY_TYPE                794956 non-null  object 
 10  LANE_CNT                       199006 non-null  float64
 11  ALIGNMENT                      794956 non-null  object 
 12  ROADWAY_SURFACE_COND          

In [31]:
print('This data has {} rows and {} columns'.format(df.shape[0], df.shape[1]))

This data has 794956 rows and 48 columns


In [32]:
print("Dataset missing values: \n", df.isna().sum())
plt.show()

Dataset missing values: 
 CRASH_RECORD_ID                       0
CRASH_DATE_EST_I                 735350
CRASH_DATE                            0
POSTED_SPEED_LIMIT                    0
TRAFFIC_CONTROL_DEVICE                0
DEVICE_CONDITION                      0
WEATHER_CONDITION                     0
LIGHTING_CONDITION                    0
FIRST_CRASH_TYPE                      0
TRAFFICWAY_TYPE                       0
LANE_CNT                         595950
ALIGNMENT                             0
ROADWAY_SURFACE_COND                  0
ROAD_DEFECT                           0
REPORT_TYPE                       23258
CRASH_TYPE                            0
INTERSECTION_RELATED_I           612741
NOT_RIGHT_OF_WAY_I               758229
HIT_AND_RUN_I                    546330
DAMAGE                                0
DATE_POLICE_NOTIFIED                  0
PRIM_CONTRIBUTORY_CAUSE               0
SEC_CONTRIBUTORY_CAUSE                0
STREET_NO                             0
STREET_DIRECTI

In [34]:
# Danh sách các cột cần xóa
columns_to_drop = [
    "CRASH_RECORD_ID", "CRASH_DATE_EST_I","CRASH_DATE", "LANE_CNT", "REPORT_TYPE",
    "INTERSECTION_RELATED_I", "NOT_RIGHT_OF_WAY_I", "HIT_AND_RUN_I",
    "DATE_POLICE_NOTIFIED", "SEC_CONTRIBUTORY_CAUSE", "STREET_NO",
    "STREET_DIRECTION", "STREET_NAME", "BEAT_OF_OCCURRENCE",
    "PHOTOS_TAKEN_I", "STATEMENTS_TAKEN_I", "DOORING_I", "WORK_ZONE_I",
    "WORK_ZONE_TYPE", "WORKERS_PRESENT_I", "INJURIES_TOTAL", "INJURIES_FATAL",
    "INJURIES_INCAPACITATING", "INJURIES_NON_INCAPACITATING",
    "INJURIES_REPORTED_NOT_EVIDENT", "INJURIES_NO_INDICATION", "INJURIES_UNKNOWN", "LATITUDE", "LONGITUDE", "LOCATION",  "MOST_SEVERE_INJURY"
]

# Xóa các cột, bỏ qua các cột không tồn tại
df = df.drop(columns=columns_to_drop, errors='ignore')

# Kiểm tra kết quả
print("Danh sách cột sau khi xóa:")
print(df.columns)


Danh sách cột sau khi xóa:
Index(['POSTED_SPEED_LIMIT', 'TRAFFIC_CONTROL_DEVICE', 'DEVICE_CONDITION',
       'WEATHER_CONDITION', 'LIGHTING_CONDITION', 'FIRST_CRASH_TYPE',
       'TRAFFICWAY_TYPE', 'ALIGNMENT', 'ROADWAY_SURFACE_COND', 'ROAD_DEFECT',
       'CRASH_TYPE', 'DAMAGE', 'PRIM_CONTRIBUTORY_CAUSE', 'NUM_UNITS',
       'CRASH_HOUR', 'CRASH_DAY_OF_WEEK', 'CRASH_MONTH'],
      dtype='object')


In [35]:
unique_count = df['CRASH_TYPE'].nunique()
print(f"Số lượng giá trị khác nhau trong CRASH_TYPE: {unique_count}")

value_counts = df['CRASH_TYPE'].value_counts()
print("\nSố lần xuất hiện của từng giá trị trong CRASH_TYPE:")
print(value_counts)


Số lượng giá trị khác nhau trong CRASH_TYPE: 2

Số lần xuất hiện của từng giá trị trong CRASH_TYPE:
CRASH_TYPE
NO INJURY / DRIVE AWAY              583177
INJURY AND / OR TOW DUE TO CRASH    211779
Name: count, dtype: int64


In [37]:

# Loại bỏ các hàng null
df = df.dropna()

In [38]:

# Thay đổi giá trị của CRASH_TYPE
df['CRASH_TYPE'] = df['CRASH_TYPE'].apply(lambda x: 1 if x == 'INJURY AND / OR TOW DUE TO CRASH' else 0)

# Mã hóa các cột dạng chuỗi thành số (các cột khác ngoài CRASH_TYPE)
categorical_cols = ['POSTED_SPEED_LIMIT', 'TRAFFIC_CONTROL_DEVICE', 'DEVICE_CONDITION', 'WEATHER_CONDITION', 
                    'LIGHTING_CONDITION', 'FIRST_CRASH_TYPE', 'TRAFFICWAY_TYPE', 'ALIGNMENT', 
                    'ROADWAY_SURFACE_COND', 'ROAD_DEFECT', 'DAMAGE', 'PRIM_CONTRIBUTORY_CAUSE', 
                    'CRASH_HOUR', 'CRASH_DAY_OF_WEEK', 'CRASH_MONTH']

label_encoder = LabelEncoder()
for col in categorical_cols:
    df[col] = label_encoder.fit_transform(df[col])


# Tách X và y
X = df.drop('CRASH_TYPE', axis=1)  # X là dữ liệu đầu vào
y = df['CRASH_TYPE']  # y là nhãn mục tiêu

# Chia dữ liệu thành bộ huấn luyện và bộ kiểm tra (70% huấn luyện, 30% kiểm tra)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tính toán trọng số lớp để xử lý mất cân bằng lớp trong dữ liệu
classes = np.unique(y_train)  # Lấy danh sách các lớp có trong y_train
class_weights = compute_class_weight('balanced', classes=classes, y=y_train)  # Tính toán trọng số lớp
class_weight_dict = dict(zip(classes, class_weights))  # Chuyển trọng số lớp thành từ điển

# Định nghĩa các tham số cho GridSearchCV
param_grid_rf = {
    'n_estimators': [50, 100, 200],  # Số lượng cây trong rừng
    'max_depth': [3, 6, 10],          # Độ sâu tối đa của cây
    'min_samples_split': [2, 5, 10],  # Số mẫu tối thiểu để chia một node
    'min_samples_leaf': [1, 2, 4],    # Số mẫu tối thiểu để tạo một lá
    'bootstrap': [True, False]        # Sử dụng bootstrap cho việc huấn luyện các cây
}

# Tạo đối tượng GridSearchCV cho RandomForest
model = RandomForestClassifier(class_weight=class_weight_dict, random_state=42)
grid_search_rf = GridSearchCV(estimator=model, param_grid=param_grid_rf, cv=3, n_jobs=-1, verbose=2)

# Huấn luyện với GridSearchCV
grid_search_rf.fit(X_train, y_train)

# In ra các tham số tốt nhất
print("Best parameters found: ", grid_search_rf.best_params_)

# Dự đoán với mô hình tốt nhất
y_pred_best = grid_search_rf.best_estimator_.predict(X_test)

# Hiển thị các báo cáo đánh giá mô hình tốt nhất
print("\nClassification Report for Best Model:")
print(classification_report(y_test, y_pred_best))

# Confusion Matrix cho mô hình tốt nhất
print("\nConfusion Matrix for Best Model:")
print(confusion_matrix(y_test, y_pred_best))

# ROC AUC Score cho mô hình tốt nhất
y_prob_best = grid_search_rf.best_estimator_.predict_proba(X_test)[:, 1]
roc_auc_best = roc_auc_score(y_test, y_prob_best)
print("\nROC AUC Score for Best Model:")
print(roc_auc_best)

# Log Loss cho mô hình tốt nhất
log_loss_value_best = log_loss(y_test, y_prob_best)
print("\nLog Loss for Best Model:")
print(log_loss_value_best)

# Matthews Correlation Coefficient (MCC) cho mô hình tốt nhất
mcc_value_best = matthews_corrcoef(y_test, y_pred_best)
print("\nMatthews Correlation Coefficient (MCC) for Best Model:")
print(mcc_value_best)

# Balanced Accuracy cho mô hình tốt nhất
balanced_accuracy_value_best = balanced_accuracy_score(y_test, y_pred_best)
print("\nBalanced Accuracy for Best Model:")
print(balanced_accuracy_value_best)


Fitting 3 folds for each of 162 candidates, totalling 486 fits


C:\Users\DELL\anaconda3\envs\THKTDL\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best parameters found:  {'bootstrap': True, 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 50}

Classification Report for Best Model:
              precision    recall  f1-score   support

           0       0.89      0.78      0.83    116676
           1       0.54      0.72      0.62     42316

    accuracy                           0.76    158992
   macro avg       0.71      0.75      0.72    158992
weighted avg       0.79      0.76      0.77    158992


Confusion Matrix for Best Model:
[[90563 26113]
 [11717 30599]]

ROC AUC Score for Best Model:
0.8243634085158027

Log Loss for Best Model:
0.5183585034297793

Matthews Correlation Coefficient (MCC) for Best Model:
0.46065010731187944

Balanced Accuracy for Best Model:
0.7496496446543577
